# Function Definition and packages download

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.1 MB/s eta 0:00:00


In [ ]:
#! pip install pandas==1.5.3

In [ ]:
import json

def read_json_keys(filepath):
    with open(filepath) as f:
        data = json.load(f)
    return list(data.keys())

def read_json(filepath):
  with open(filepath) as f:
        return json.load(f)


def save_dict_to_json_file(data, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


In [ ]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

import re

def extract_decimal(s):
    # Use a regular expression to search for a decimal number
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    if match:
        return float(match.group(0))
    else:
        return None

        
import statistics
from typing import List

def calculate_statistics(numbers: List[float]) -> dict:
    statistics_data = {}
    statistics_data['mean'] = statistics.mean(numbers)
    statistics_data['median'] = statistics.median(numbers)
    try:
        statistics_data['mode'] = statistics.mode(numbers)
    except statistics.StatisticsError:
        statistics_data['mode'] = None
    statistics_data['stdev'] = statistics.stdev(numbers)
    statistics_data['variance'] = statistics.variance(numbers)
    return statistics_data


In [ ]:
import os

def read_documents_in_folder(folder_path: str) -> list:
    document_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
              document_path = os.path.join(root, file)
              document_paths.append(document_path)
    return document_paths

In [ ]:
import pandas as pd
import pickle

def create_dataset_pickle(lista, path):
    dataset = pd.DataFrame(lista, columns=['text'])
    with open(path, 'wb') as f:
        pickle.dump(dataset, f)


# Filter the results

In [ ]:
from scipy.stats import pearsonr

path_risultati_dir=""
path_dir=""

path_completo=path_dir
path_risultati= path_risultati_dir
with open(path_completo) as f:
  data = json.load(f)
new_risultati={}
for k in read_json_keys(path_completo):
  print(k)
  smear=[]
  wn=[]
  new_risultati[k]={}
  new_risultati[k]["Risultati totali"]=[]
  count=0
  #print(data[k])
  for i in data[k]['risultati_totali']:
    if is_number(i[0]) or is_number(i[2]):
      pass
    else:
      if i not in new_risultati[k]["Risultati totali"] and  i[0] != i[2] and len(i[0])>1 and len(i[2])>1:
        new_risultati[k]["Risultati totali"].append(i)
        count+=1
        wn.append(extract_decimal(i[4]))
        smear.append(extract_decimal(i[5]))
  new_risultati[k]["Numero coppie"]= count
  if len(wn)>1:
    new_risultati[k]["Wordnet"]= calculate_statistics(wn)
    new_risultati[k]["Smear"]= calculate_statistics(smear)
    correlation, _ = pearsonr(wn, smear)
    new_risultati[k]["Correlation"]=correlation

save_dict_to_json_file(new_risultati, path_risultati)

# Analyse the results

In [ ]:
from collections import OrderedDict
from random import sample
import matplotlib.pyplot as plt

def plot_line_from_dict(data_dict, path):
    """
    Plots a line graph from a dictionary where each key is on the x-axis and its value is on the y-axis.
    """
    # Get the keys and values from the dictionary
    x_values = list(data_dict.keys())
    y_values = list(data_dict.values())

    # Set up the figure and axis
    #fig, ax = plt.subplots()
    fig, ax = plt.subplots(figsize=(15, 5))

    # Plot the line
    ax.plot(x_values, y_values)
    plt.xticks(rotation=90)


    # Show the plot
    fig.savefig(path)
    plt.show()


path=""
path_model=path+"" #folder where the data of that model are stored
data=read_json(path_model+"") #Where the results are stored
chiavi=read_json(path+"")#Where the datasets are stored
path_risultati=path_model+"risultati_anto.json"
diag_path="diag_anto.json"
tot=len(chiavi)
print(tot)
risultati={}

risultati={}
risultati["Total"]=0
risultati["Top 5"]=0
risultati["Top Self"]=0


visti_tot=[]
for h in data:
  visti=[]
  risultati[h]={}
  #visti=[]
  #print(h)
  risultati[h]["Numero coppie heads"]=len(data[h]["risultati_totali"])
  risultati[h]["Numero"]=0
  risultati[h]["Corrette"]=[]
  for t in data[h]:
    if "train" in t:
      for chiave in chiavi:
        #print(chiave)
        frase_chiave=chiavi[chiave].lower().strip()
        if frase_chiave[-1]!=".":
          frase_chiave+="."     
        if frase_chiave == data[h][t]["sentence"].lower().strip():
          #print(p, ": ",chiavi[chiave][p].lower().strip(), "      ",data[h][t]["sentence"].lower().strip() )
          for r in data[h][t]["risultati"]:
            c=chiave.split(", ")#", "
            if (r[0].lower() == c[0].lower() and r[2].lower() in c[1].lower()) or (r[0].lower() in c[1].lower() and r[2].lower()==c[0].lower()): #Vedi linea sopra per errore
              #print(h,": ",r[0], r[2], "---", c)
              if ((c[0].lower(),c[1].lower()) not in visti):
                risultati[h]["Numero"]+=1
                risultati[h]["Corrette"].append(c)
                visti.append((c[0].lower(),c[1].lower()))
                if ((c[0].lower(),c[1].lower()) not in visti_tot):
                  visti_tot.append((c[0].lower(),c[1].lower()))
                  risultati["Totali"]+=1


  if risultati[h]["Numero coppie heads"]==0:
      risultati[h]["Precision"]=0
  else:
    risultati[h]["Precision"]=risultati[p][h]["Numero"]/risultati[h]["Numero coppie heads"]
  risultati[h]["Recall"]=risultati[h]["Numero"]/tot
  if risultati[h]["Precision"] ==0 or risultati[h]["Recall"]==0:
    risultati[h]["F1"]=0
  else:
    risultati[h]["F1"]=2*risultati[h]["Precision"]*risultati[h]["Recall"]/(risultati[h]["Precision"]+risultati[h]["Recall"])

  lista=[]
  diag=read_json(path+diag_path)
  keys=list(diag.keys())
  #rand=sample(keys,20)
  for k in keys[:20]:
    #print(k)
    for coppia in risultati[k]["Corrette"]:
      if coppia not in lista:
        lista.append(coppia)
  risultati["Top Diagonal"]=len(lista)


  show_dict={}
  for d in diag:
    show_dict[d]=risultati[p][d]["F1"]
  ord=dict(OrderedDict(sorted(show_dict.items(), key=lambda t: t[1], reverse=True)))
  keys=list(ord.keys())[:20]
  #print(keys)
  lista=[]
  for k in keys:
    print(k, ord[k])
    for coppia in risultati[p][k]["Corrette"]:
      if coppia not in lista:
        lista.append(coppia)
  risultati[p]["Top 5"]=len(lista)
  print("\n\n\n")

save_dict_to_json_file(risultati, path_risultati)